# Battle of the Neighborhoods
# Hampden v Murrayfield
# 2018

## Hampden Stadium is in the Mount Florida area of Glasgow; Murrayfield Stadium is in the Roseburn Area of Edinburgh...

### Let Battle Commence...

### Import necessary Libraries

In [10]:
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values
import requests # library to handle requests
import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import numpy as np # library to handle data in a vectorized manner
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

# matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from scikit-learn
from sklearn.cluster import KMeans

!pip install folium
import folium # map rendering library

print ('Folium installed')
print ('Libraries imported.')

    100% |████████████████████████████████| 92kB 6.3MB/s eta 0:00:01
Requirement not upgraded as not directly required: jinja2 in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from folium)
Requirement not upgraded as not directly required: six in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from folium)
Requirement not upgraded as not directly required: numpy in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from folium)
Requirement not upgraded as not directly required: requests in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from folium)
Requirement not upgraded as not directly required: MarkupSafe>=0.23 in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from jinja2->folium)
Requirement not upgraded as not directly required: chardet<3.1.0,>=3.0.2 in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from requests->folium)
Requirement not upgraded as not directly required: idna<2.7,>=2.5 in /opt/conda/envs/DSX-Python35/

<a id='item1'></a>

## 1. Download and Explore Dataset

#### Download data from IBM server

In [12]:
# The code was removed by Watson Studio for sharing.

,postal code,borough,neighborhood,latitude,longitude
0,G42 9DD,Glasgow,Mount Florida,55.8258,-4.2520
1,EH12 5PJ,Edinburgh,Roseburn,55.9422,-3.2408


#### Create a map of Scotland with neighborhoods superimposed on top

In [25]:
# create map of Scotland using latitude and longitude values
map_scotland = folium.Map(location=[55.9024, -3.7431], zoom_start=9.5)

# add markers to map
for lat, lng, postal_code, label_list in zip(df_data_1['latitude'], df_data_1['longitude'], df_data_1['postal code'], df_data_1['neighborhood']):
    label = str(postal_code) + ', ' + (label_list)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_scotland)  
    
map_scotland

#### Define Foursquare Credentials and Version

Make sure that you have created a Foursquare developer account and have your credentials handy

In [26]:
# The code was removed by Watson Studio for sharing.

#### Let's explore the first neighborhood in our dataframe

Get the neighborhood's name

In [27]:
df_data_1.loc[0, 'neighborhood']

'Mount Florida'

Get the neighborhood's latitude and longitude values

In [36]:
neighborhood_latitude = df_data_1.loc[0, 'latitude'] # neighborhood latitude value
neighborhood_longitude = df_data_1.loc[0, 'longitude'] # neighborhood longitude value

neighborhood_name = df_data_1.loc[0, 'neighborhood'] # neighborhood name

print ('Latitude and longitude values of ')
print (neighborhood_name)
print(neighborhood_latitude)
print(neighborhood_longitude)

Latitude and longitude values of 
Mount Florida
55.8258
-4.252


#### Now, let's get the top 100 venues that are within a radius of 1000 meters.

First, let's create the GET request URL

In [37]:
radius = 1000 # define radius

# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?&client_id=04AX30N3OMVLA4D3KIM5EN5HA34ZSOGVY14QV5Y2V3HUAMDF&client_secret=MMKHPKM2SBL5LSDP0F1FXSFDC2CYRDPNA1ZS45WK2JBMEVI1&v=20171018&ll=55.8258,-4.252&radius=1000&limit=100'

Send the GET request and examine the resutls

In [38]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5c16b9166a60717235e6e5e9'},
 'response': {'groups': [{'items': [{'reasons': {'count': 0,
       'items': [{'reasonName': 'globalInteractionReason',
         'summary': 'This spot is popular',
         'type': 'general'}]},
      'referralId': 'e-0-4b69ceb7f964a52075b52be3-0',
      'venue': {'beenHere': {'count': 0,
        'lastCheckinExpiredAt': 0,
        'marked': False,
        'unconfirmedCount': 0},
       'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/arts_entertainment/stadium_soccer_',
          'suffix': '.png'},
         'id': '4bf58dd8d48988d188941735',
         'name': 'Soccer Stadium',
         'pluralName': 'Soccer Stadiums',
         'primary': True,
         'shortName': 'Soccer'}],
       'contact': {},
       'hereNow': {'count': 0, 'groups': [], 'summary': 'Nobody here'},
       'id': '4b69ceb7f964a52075b52be3',
       'location': {'address': 'Letherby Dr',
        'cc': 'GB',
        'city': 'Glasgow

From Foursquare presentation, we know that all the information is in the *items* key. Before we proceed, let's borrow the **get_category_type** function from the Foursquare presentation

In [39]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name'].encode('ascii', errors='ignore')

Now we are ready to clean the json and structure it into a *pandas* dataframe

In [42]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Hampden Park,b'Soccer Stadium',55.825563,-4.251821
1,Sapori D'italia,b'Italian Restaurant',55.826050,-4.257485
2,Hooked Fish & Chips,b'Fish & Chips Shop',55.827126,-4.259064
3,Tinto Tapas Bar,b'Tapas Restaurant',55.826488,-4.265234
4,The Scottish Football Museum,b'History Museum',55.825231,-4.252044


And how many venues were returned by Foursquare?

In [44]:
print ('{} venues were returned by Foursquare'.format(nearby_venues.shape[0]))

25 venues were returned by Foursquare


<a id='item2'></a>

## 2. Explore Neighborhoods around Hampden and Murrayfield

#### Let's create a function to repeat the same process to all the neighborhoods of Hampden and Murrayfield

In [45]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results[0]['items']])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['neighborhood', 
                  'neighborhood latitude', 
                  'neighborhood longitude', 
                  'venue', 
                  'venue latitude', 
                  'venue longitude', 
                  'venue category']
    
    return(nearby_venues)

#### Now let's run the function on each neighborhood

In [46]:
scot_venues = getNearbyVenues(names=df_data_1.neighborhood,
                              latitudes=df_data_1.latitude,
                              longitudes=df_data_1.longitude,
                              radius=1000)

Mount Florida
Roseburn


#### Let's check the size of the resulting dataframe

In [48]:
print (scot_venues.shape)
scot_venues.head()

(67, 7)


,neighborhood,neighborhood latitude,neighborhood longitude,venue,venue latitude,venue longitude,venue category
0,Mount Florida,55.8258,-4.252,Hampden Park,55.825563,-4.251821,Soccer Stadium
1,Mount Florida,55.8258,-4.252,Sapori D'italia,55.826050,-4.257485,Italian Restaurant
2,Mount Florida,55.8258,-4.252,Hooked Fish & Chips,55.827126,-4.259064,Fish & Chips Shop
3,Mount Florida,55.8258,-4.252,Tinto Tapas Bar,55.826488,-4.265234,Tapas Restaurant
4,Mount Florida,55.8258,-4.252,The Scottish Football Museum,55.825231,-4.252044,History Museum


#### Let's check how many venues were returned for each neighborhood

In [49]:
scot_venues.groupby('neighborhood').count()

,neighborhood latitude,neighborhood longitude,venue,venue latitude,venue longitude,venue category
neighborhood,,,,,,
Mount Florida,25,25,25,25,25,25
Roseburn,42,42,42,42,42,42


#### Let's note how many unique categories can be curated from all the returned venues

In [51]:
print ('{} uniques categories'.format(len(scot_venues['venue category'].unique())))

41 uniques categories


<a id='item3'></a>

## 3. Analyze Each Neighborhood

In [52]:
# one hot encoding
scot_onehot = pd.get_dummies(scot_venues[['venue category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
scot_onehot['neighborhood'] = scot_venues['neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [scot_onehot.columns[-1]] + list(scot_onehot.columns[:-1])
scot_onehot = scot_onehot[fixed_columns]

scot_onehot.head()

,neighborhood,Bakery,Bar,Bookstore,Bowling Alley,Bus Stop,Café,Chinese Restaurant,Clothing Store,Coffee Shop,Convenience Store,Curling Ice,Farm,Fast Food Restaurant,Fish & Chips Shop,Furniture / Home Store,Gas Station,Gastropub,Grocery Store,Gym,Hardware Store,History Museum,Hotel,Indian Restaurant,Italian Restaurant,Park,Pet Store,Pharmacy,Pizza Place,Playground,Pub,Rugby Stadium,Sandwich Place,Skate Park,Skating Rink,Soccer Field,Soccer Stadium,Sports Bar,Supermarket,Tapas Restaurant,Trail,Tram Station
0,Mount Florida,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
1,Mount Florida,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Mount Florida,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Mount Florida,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
4,Mount Florida,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


And let's examine the new dataframe size

In [54]:
scot_onehot.shape

(67, 42)

#### Next, let's group rows by neighborhood

In [55]:
scot_grouped = scot_onehot.groupby('neighborhood').mean().reset_index()
scot_grouped

,neighborhood,Bakery,Bar,Bookstore,Bowling Alley,Bus Stop,Café,Chinese Restaurant,Clothing Store,Coffee Shop,Convenience Store,Curling Ice,Farm,Fast Food Restaurant,Fish & Chips Shop,Furniture / Home Store,Gas Station,Gastropub,Grocery Store,Gym,Hardware Store,History Museum,Hotel,Indian Restaurant,Italian Restaurant,Park,Pet Store,Pharmacy,Pizza Place,Playground,Pub,Rugby Stadium,Sandwich Place,Skate Park,Skating Rink,Soccer Field,Soccer Stadium,Sports Bar,Supermarket,Tapas Restaurant,Trail,Tram Station
0,Mount Florida,0.04000,0.04000,0.04,0.00000,0.00000,0.08000,0.00000,0.00000,0.04000,0.00000,0.00000,0.00000,0.00000,0.04000,0.00000,0.00000,0.00000,0.080000,0.04000,0.00000,0.04,0.040000,0.08,0.08,0.08000,0.00000,0.000000,0.04,0.00000,0.04000,0.00000,0.04000,0.00000,0.00000,0.04000,0.04000,0.000000,0.04000,0.04,0.00000,0.000000
1,Roseburn,0.02381,0.02381,0.00,0.02381,0.02381,0.02381,0.02381,0.02381,0.02381,0.02381,0.02381,0.02381,0.02381,0.02381,0.02381,0.02381,0.02381,0.047619,0.02381,0.02381,0.00,0.095238,0.00,0.00,0.02381,0.02381,0.047619,0.00,0.02381,0.02381,0.02381,0.02381,0.02381,0.02381,0.02381,0.02381,0.047619,0.02381,0.00,0.02381,0.047619


#### Let's confirm the new size

In [56]:
scot_grouped.shape

(2, 42)

#### Let's print each neighborhood along with the top 5 most common venues

In [58]:
num_top_venues = 5

for hood in scot_grouped['neighborhood']:
    print("----"+hood+"----")
    temp = scot_grouped.loc[scot_grouped['neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.loc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Mount Florida----
                venue  freq
0       Grocery Store  0.08
1                Café  0.08
2                Park  0.08
3  Italian Restaurant  0.08
4   Indian Restaurant  0.08


----Roseburn----
           venue  freq
0          Hotel  0.10
1   Tram Station  0.05
2  Grocery Store  0.05
3     Sports Bar  0.05
4       Pharmacy  0.05




#### Let's put that into a *pandas* dataframe

First, let's write a function to sort the venues in descending order

In [59]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Now let's create the new dataframe and display the top 10 venues for each neighborhood

In [60]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['neighborhood'] = scot_grouped['neighborhood']

for ind in np.arange(scot_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(scot_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Mount Florida,Café,Grocery Store,Indian Restaurant,Italian Restaurant,Park,History Museum,Pizza Place,Bar,Bookstore,Coffee Shop
1,Roseburn,Hotel,Tram Station,Sports Bar,Pharmacy,Grocery Store,Bus Stop,Café,Chinese Restaurant,Clothing Store,Coffee Shop


<a id='item4'></a>

## 4. Cluster Neighborhoods

Run *k*-means to cluster the neighborhood into 2 clusters

In [62]:
# set number of clusters
kclusters = 2

scot_grouped_clustering = scot_grouped.drop('neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(scot_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 1], dtype=int32)

Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood

In [64]:
scot_merged = df_data_1

# add clustering labels
scot_merged['cluster_labels'] = kmeans.labels_

# merge scot_grouped with df data 1 to add latitude/longitude for each neighborhood
scot_merged = scot_merged.join(neighborhoods_venues_sorted.set_index('neighborhood'), on='neighborhood')

scot_merged.head() # check the last columns!

,postal code,borough,neighborhood,latitude,longitude,cluster_labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,G42 9DD,Glasgow,Mount Florida,55.8258,-4.2520,0,Café,Grocery Store,Indian Restaurant,Italian Restaurant,Park,History Museum,Pizza Place,Bar,Bookstore,Coffee Shop
1,EH12 5PJ,Edinburgh,Roseburn,55.9422,-3.2408,1,Hotel,Tram Station,Sports Bar,Pharmacy,Grocery Store,Bus Stop,Café,Chinese Restaurant,Clothing Store,Coffee Shop


Finally, let's visualize the resulting clusters

In [68]:
# create map
map_clusters = folium.Map(location=[55.9024, -3.7431], zoom_start=9.5)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(scot_merged['latitude'], scot_merged['longitude'], scot_merged['neighborhood'], scot_merged['cluster_labels']):
    label = str(poi) + ' Cluster ' + str(cluster)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## 5. Examine Clusters

#### Cluster 1

In [72]:
scot_merged.loc[scot_merged['cluster_labels'] == 0]

,postal code,borough,neighborhood,latitude,longitude,cluster_labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,G42 9DD,Glasgow,Mount Florida,55.8258,-4.252,0,Café,Grocery Store,Indian Restaurant,Italian Restaurant,Park,History Museum,Pizza Place,Bar,Bookstore,Coffee Shop


#### Cluster 2

In [73]:
scot_merged.loc[scot_merged['cluster_labels'] == 1]

,postal code,borough,neighborhood,latitude,longitude,cluster_labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,EH12 5PJ,Edinburgh,Roseburn,55.9422,-3.2408,1,Hotel,Tram Station,Sports Bar,Pharmacy,Grocery Store,Bus Stop,Café,Chinese Restaurant,Clothing Store,Coffee Shop


  # Thank you for reviewing!  